In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
from tqdm import tqdm
HOME = expanduser("~")
import os, sys
import SimpleITK as sitk
%load_ext autoreload
%autoreload 2

In [6]:
animal = 'DK63'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')
ELASTIX = os.path.join(DIR, 'elastix')
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.registration.utilities_registration import register_test, register, register_simple

In [3]:
fixed_index = str(225).zfill(3)
moving_index = str(226).zfill(3)

In [5]:
test_transform, fixed, moving, R = register_test(INPUT, fixed_index, moving_index)
testParameters = test_transform.GetParameters()
testFixedParameters = test_transform.GetFixedParameters()
# Always check the reason optimization terminated.
stop_condition = R.GetOptimizerStopConditionDescription()
iterations = R.GetOptimizerIteration()
metric_value = R.GetMetricValue()

print(f'Final metric value: {metric_value}')
print(f'Optimizer\'s stopping condition, {stop_condition}')
print('final', testParameters)
print('fixed', testFixedParameters)

RuntimeError: Exception thrown in SimpleITK ReadImage: /home/eddyod/programming/SimpleElastix/Code/IO/src/sitkImageReaderBase.cxx:99:
sitk::ERROR: The file "/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK61/preps/CH1/thumbnail_cleaned/226.tif" does not exist.

In [ ]:
%%time
# register
#final_transform = register(INPUT, fixed_index, moving_index)
files = sorted(os.listdir(INPUT))
for f in range(len(files) - 1):
    fixed_index = str(f).zfill(3)
    moving_index = str(f+1).zfill(3)
    outdir = os.path.join(ELASTIX, f'{moving_index}_to_{fixed_index}')
    os.makedirs(outdir, exist_ok=True)
    outfile = os.path.join(outdir, 'TransformParameters.0.txt')
    
    if os.path.exists(outfile):
        continue
    
    final_transform = register(INPUT, fixed_index, moving_index)
    R, xshift, yshift = final_transform.GetParameters()
    midx, midy = final_transform.GetFixedParameters()

    f = open(outfile, "a")
    f.write(f"(TransformParameters {R} {xshift} {yshift})\n")
    f.write(f"(CenterOfRotationPoint {midx} {midy})\n")
    f.write("(Spacing 1.0 1.0)\n")
    f.close()


In [ ]:
f = sitk.SimpleElastix()
#test_transform = register_simple(INPUT, fixed_index, moving_index)


In [ ]:
moving_resampled = sitk.Resample(moving, fixed, final_transform, sitk.sitkLinear, 0.0, moving.GetPixelID())
fig = plt.figure(figsize=(15,8))
plt.imshow(sitk.GetArrayViewFromImage(fixed), cmap='gray')
plt.grid()
plt.title('fixed image', fontsize=10)

fig = plt.figure(figsize=(15, 8))
plt.imshow(sitk.GetArrayViewFromImage(moving_resampled), cmap='gray')
plt.grid()
plt.title('resampled image')

fig = plt.figure(figsize=(15, 8))
plt.title('moving image')
plt.grid()
plt.imshow(sitk.GetArrayViewFromImage(moving), cmap='gray')
    

In [ ]:
#geometry Final metric value: -0.3050850951798145
#Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Step too small after 69 iterations. Current step (6.10352e-05) is less than minimum step (0.0001).
#rotation, xshift, yshift (0.002372645049694355, 1.06484770727788, -29.715144692551913)

#moments Final metric value: -0.31838088083076627
#Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Maximum number of iterations (80) exceeded.
#rotation, xshift, yshift (-0.0034537801668540033, -28.17482735969523, -45.74971520579796)

#moments, with no Setoptscales Final metric value: -0.31014666630255233
#Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Step too small after 26 iterations. .
#rotation, xshift, yshift (-0.06870509561607951, -4.210244784366168, -35.41774791170732)

# means square
#rotation, xshift, yshift (-0.004593785628556457, -28.54399266359482, -45.8018880776268)

#simialarity Final metric value: -0.3653938756091227
#Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Maximum number of iterations (80) exceeded.
#rotation, xshift, yshift (0.9354990407253294, 0.006315032681630877, -47.83047698059379, -41.41794215960105)

In [ ]:
0.9354990407253294 * 0.006315032681630877